## 指令遵从数据集

### 加载数据

In [3]:
import datasets

raw_dataset = datasets.load_dataset("json", data_files="data/alpaca_gpt4_data_zh.json")
print(raw_dataset)

/home/chenshaohon/miniconda3/envs/em-llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 42677
    })
})


### 读取一条数据展示结构

In [4]:
raw_dataset["train"][0]

{'instruction': '保持健康的三个提示。',
 'input': '',
 'output': '以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。'}

In [6]:
import json
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("weights/glm-4-9b-hf")
tokenizer.chat_template = "[gMASK]<sop>{% for item in messages %}{% if item['tools'] is defined %}<|system|>\n你是一个名为 ChatGLM 的人工智能助手。你是基于智谱AI训练的语言模型 GLM-4 模型开发的，你的任务是针对用户的问题和要求提供适当的答复和支持。\n\n# 可用工具{% set tools = item['tools'] %}{% for tool in tools %}{% if tool['type'] == 'function' %}\n\n## {{ tool['function']['name'] }}\n\n{{ tool['function'] | tojson(indent=4) }}\n在调用上述函数时，请使用 Json 格式表示调用的参数。{% elif tool['type'] == 'python' %}\n\n## python\n\n当你向 `python` 发送包含 Python 代码的消息时，该代码将会在一个有状态的 Jupyter notebook 环境中执行。\n`python` 返回代码执行的输出，或在执行 60 秒后返回超时。\n`/mnt/data` 将会持久化存储你的文件。在此会话中，`python` 无法访问互联网。不要使用 `python` 进行任何网络请求或者在线 API 调用，这些在线内容的访问将不会成功。{% elif tool['type'] == 'simple_browser' %}\n\n## simple_browser\n\n你可以使用 `simple_browser` 工具。该工具支持以下函数：\n`search(query: str, recency_days: int)`：使用搜索引擎进行查询并显示结果，可以使用 `recency_days` 参数控制搜索内容的时效性。\n`mclick(ids: list[int])`：获取一系列指定 id 的页面内容。每次调用时，须选择3-10个页面。选择多个角度的页面，同时尽可能选择可信任的信息来源。考虑到部分页面是无法加载的，你也可以多打开一些可能有用的页面而不用担心内容过多。\n`open_url(url: str)`：打开指定的 URL。\n\n使用 `【{引用 id}†{引用文本}】` 来引用内容。\n\n操作步骤：1. 使用 `search` 来获得信息列表; 2. 使用 `mclick` 来获取指定 ID 页面的内容; 3. 根据获得的内容进行回复。在回复中应当引用信息来源。\n 如果用户提供了 URL，也可以用 `open_url` 直接打开页面。\n如果初次搜索结果没有找到合适的信息，也可以再次使用 `search` 进行搜索。{% elif tool['type'] == 'cogview' %}\n\n## cogview\n\n如果用户的请求中包含了对图像的描述，你可以使用 `cogview` 来生成图像并展示给用户。你需要向 `cogview` 发送图像描述，规则：\n- 发送给 `cogview` 的消息必须使用英语。用户的中文描述必须完全翻译为英语。\n- 应当尽可能详细地描述图像生成的需求，需求描述约 100 英文单词。\n- 保持用户原始描述的意图。不要虚构内容或者没见过的人物。\n- 如无特殊说明，所在地为中国，持有中国立场并遵循中国社会主义价值观。{% endif %}{% endfor %}{% endif %}{% if item['content'] %}<|{{ item['role'] }}|>{{ item['metadata'] }}\n{{ item['content'] }}{% endif %}{% endfor %}{% if add_generation_prompt %}<|assistant|>{% endif %}"


def formatting_func(example):
    """
    process data format
    """
    prompt = example["instruction"]
    if len(example["input"]) != 0:
        prompt += "\n\n" + example["input"]
    conversations = [
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": example["output"]},
    ]
    output_text = tokenizer.apply_chat_template(
        conversation=conversations, tokenize=False
    )
    return output_text

print(formatting_func(raw_dataset["train"][0]))

[gMASK]<sop><|user|>
保持健康的三个提示。<|assistant|>
以下是保持健康的三个提示：

1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。

2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。

3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。


## 工具数据集

### 加载数据

In [1]:
import datasets

raw_dataset=datasets.load_dataset("json", data_files="data/glaive_toolcall_zh_1k.json")
print(raw_dataset)

/home/chenshaohon/miniconda3/envs/em-llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['conversations', 'tools'],
        num_rows: 1000
    })
})


### 打印一条数据

In [2]:
raw_dataset["train"][0]

{'conversations': [{'from': 'human', 'value': '你好，我需要一个1到100之间的随机数。'},
  {'from': 'function_call',
   'value': '{"name": "generate_random_number", "arguments": {"min": 1, "max": 100}}'},
  {'from': 'observation', 'value': '{"number": 57}'},
  {'from': 'gpt', 'value': '生成的随机数在1到100之间，是57。'},
  {'from': 'human', 'value': '好的，可以。这次生成一个长度在200到300之间的句子。'},
  {'from': 'function_call',
   'value': '{"name": "generate_random_number", "arguments": {"min": 200, "max": 300}}'},
  {'from': 'observation', 'value': '{"number": 267}'},
  {'from': 'gpt', 'value': '生成的随机数在200到300之间，是267。'},
  {'from': 'human', 'value': '谢谢，这些就是我需要的全部。'},
  {'from': 'gpt', 'value': '不客气！如果你还需要其他什么，随时问。'}],
 'tools': '[{"name": "generate_random_number", "description": "在指定范围内生成一个随机数", "parameters": {"type": "object", "properties": {"min": {"type": "integer", "description": "最小值"}, "max": {"type": "integer", "description": "最大值"}}, "required": ["min", "max"]}}]'}

### 验证格式整理

In [3]:
import json
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("weights/glm-4-9b-hf")
tokenizer.chat_template = "[gMASK]<sop>{% for item in messages %}{% if item['tools'] is defined %}<|system|>\n你是一个名为 ChatGLM 的人工智能助手。你是基于智谱AI训练的语言模型 GLM-4 模型开发的，你的任务是针对用户的问题和要求提供适当的答复和支持。\n\n# 可用工具{% set tools = item['tools'] %}{% for tool in tools %}{% if tool['type'] == 'function' %}\n\n## {{ tool['function']['name'] }}\n\n{{ tool['function'] | tojson(indent=4) }}\n在调用上述函数时，请使用 Json 格式表示调用的参数。{% elif tool['type'] == 'python' %}\n\n## python\n\n当你向 `python` 发送包含 Python 代码的消息时，该代码将会在一个有状态的 Jupyter notebook 环境中执行。\n`python` 返回代码执行的输出，或在执行 60 秒后返回超时。\n`/mnt/data` 将会持久化存储你的文件。在此会话中，`python` 无法访问互联网。不要使用 `python` 进行任何网络请求或者在线 API 调用，这些在线内容的访问将不会成功。{% elif tool['type'] == 'simple_browser' %}\n\n## simple_browser\n\n你可以使用 `simple_browser` 工具。该工具支持以下函数：\n`search(query: str, recency_days: int)`：使用搜索引擎进行查询并显示结果，可以使用 `recency_days` 参数控制搜索内容的时效性。\n`mclick(ids: list[int])`：获取一系列指定 id 的页面内容。每次调用时，须选择3-10个页面。选择多个角度的页面，同时尽可能选择可信任的信息来源。考虑到部分页面是无法加载的，你也可以多打开一些可能有用的页面而不用担心内容过多。\n`open_url(url: str)`：打开指定的 URL。\n\n使用 `【{引用 id}†{引用文本}】` 来引用内容。\n\n操作步骤：1. 使用 `search` 来获得信息列表; 2. 使用 `mclick` 来获取指定 ID 页面的内容; 3. 根据获得的内容进行回复。在回复中应当引用信息来源。\n 如果用户提供了 URL，也可以用 `open_url` 直接打开页面。\n如果初次搜索结果没有找到合适的信息，也可以再次使用 `search` 进行搜索。{% elif tool['type'] == 'cogview' %}\n\n## cogview\n\n如果用户的请求中包含了对图像的描述，你可以使用 `cogview` 来生成图像并展示给用户。你需要向 `cogview` 发送图像描述，规则：\n- 发送给 `cogview` 的消息必须使用英语。用户的中文描述必须完全翻译为英语。\n- 应当尽可能详细地描述图像生成的需求，需求描述约 100 英文单词。\n- 保持用户原始描述的意图。不要虚构内容或者没见过的人物。\n- 如无特殊说明，所在地为中国，持有中国立场并遵循中国社会主义价值观。{% endif %}{% endfor %}{% endif %}{% if item['content'] %}<|{{ item['role'] }}|>{{ item['metadata'] }}\n{{ item['content'] }}{% endif %}{% endfor %}{% if add_generation_prompt %}<|assistant|>{% endif %}"

def formatting_func(example):
    """
    process data format
    """
    tools = []
    try:
        tool_list = json.loads(example["tools"])
    except:
        tool_list = []
    for tool in tool_list:
        tools.append(
            {
                "type": "function",
                "function": {
                    "name": tool["name"],
                    "description": tool["description"],
                    "parameters": tool["parameters"],
                },
            },
        )
    conversations = [{"role": "system", "content": "", "tools": tools}]
    for chat in example["conversations"]:
        if chat["from"] == "human":
            role = "user"
        elif chat["from"] == "observation":
            role = "observation"
        else:
            role = "assistant"
        conversations.append({"role": role, "content": chat["value"]})
    return tokenizer.apply_chat_template(conversation=conversations, tokenize=False)

print(formatting_func(raw_dataset["train"][0]))

[gMASK]<sop><|system|>
你是一个名为 ChatGLM 的人工智能助手。你是基于智谱AI训练的语言模型 GLM-4 模型开发的，你的任务是针对用户的问题和要求提供适当的答复和支持。

# 可用工具
## generate_random_number

{
    "name": "generate_random_number",
    "description": "在指定范围内生成一个随机数",
    "parameters": {
        "type": "object",
        "properties": {
            "min": {
                "type": "integer",
                "description": "最小值"
            },
            "max": {
                "type": "integer",
                "description": "最大值"
            }
        },
        "required": [
            "min",
            "max"
        ]
    }
}
在调用上述函数时，请使用 Json 格式表示调用的参数。<|user|>
你好，我需要一个1到100之间的随机数。<|assistant|>
{"name": "generate_random_number", "arguments": {"min": 1, "max": 100}}<|observation|>
{"number": 57}<|assistant|>
生成的随机数在1到100之间，是57。<|user|>
好的，可以。这次生成一个长度在200到300之间的句子。<|assistant|>
{"name": "generate_random_number", "arguments": {"min": 200, "max": 300}}<|observation|>
{"number": 267}<|assistant|>
生成的随机数在200到300之间，是267。<|user|>
谢谢，这些就是我需要